<a href="https://colab.research.google.com/github/georgialoukatou/Conversational-Analysis-HAI/blob/master/HAI_ling_analysis_v.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install git+https://github.com/georgialoukatou/Cornell-Conversational-Analysis-Toolkit.git/@Hai  
#my github for extra pos and lemma analysis

import convokit
from convokit import Corpus, download

corpus = Corpus(download('subreddit-Cornell')) 
!python -m spacy download en_core_web_sm


  Cloning https://github.com/georgialoukatou/Cornell-Conversational-Analysis-Toolkit.git/ (to revision Hai) to /tmp/pip-req-build-81vty5f_
  Running command git clone -q https://github.com/georgialoukatou/Cornell-Conversational-Analysis-Toolkit.git/ /tmp/pip-req-build-81vty5f_
  Running command git checkout -b Hai --track origin/Hai
  Switched to a new branch 'Hai'
  Branch 'Hai' set up to track remote branch 'Hai' from 'origin'.
     |████████████████████████████████| 12.8MB 327kB/s 
     |████████████████████████████████| 1.4MB 47.1MB/s 
     |████████████████████████████████| 245kB 39.1MB/s 
     |████████████████████████████████| 1.1MB 49.2MB/s 
     |████████████████████████████████| 9.1MB 41.3MB/s 
     |████████████████████████████████| 460kB 39.2MB/s 
     |████████████████████████████████| 133kB 43.3MB/s 
     |████████████████████████████████| 71kB 6.8MB/s 
     |████████████████████████████████| 122kB 46.5MB/s 
  Created wheel for convokit: filename=convokit-2.4.4-cp37-none-

In [7]:
from convokit.text_processing import TextParser  
textparser = TextParser()
textparser.transform(corpus)     #perform SpaCy analysis

In [8]:
#check use of PoS
import collections
convo = corpus.random_conversation() #pick random conversation from corpus 
d = collections.defaultdict(list)

d2 = collections.defaultdict(list)
for spkr in convo.iter_speakers():
   spkr_utts = list(spkr.iter_utterances())
 #  print(spkr_utts)
   for i in spkr_utts:
      d[spkr.id].append(len(i.text.split(" "))) #counts number of words
      for j in i.meta["parsed"]: 
        d2[spkr.id].append(list())
        for k in range(len(j["toks"])) :
          d2[spkr.id][-1].append(j["toks"][k]["pos"]) #find "parsed" pos metadata for each speaker (separate by utt)

for i in range(len(d2[spkr.id])):
    d2[spkr.id][i] = collections.Counter(d2[spkr.id][i])

#d2 gives parts of speech for each utterance and each speaker)


In [9]:
print(convo)

Conversation('id': '77khqw', 'utterances': ['77khqw', 'domyj1g', 'donalum', 'dooq3q1'], 'meta': {'title': 'Does the loft need a fake to enter?', 'num_comments': 3, 'domain': 'self.Cornell', 'timestamp': 1508482721, 'subreddit': 'Cornell', 'gilded': 0, 'gildings': None, 'stickied': False, 'author_flair_text': ''})


In [10]:
#check length of utterances
import pandas as pd

df=pd.DataFrame.from_dict(d,orient='index').T #length of utterance for each speaker

print(df)

      giottoblue  [deleted]  Clue_Balls  champanova
0           34.0      105.0        97.0        15.0
1           61.0       29.0        92.0        38.0
2          114.0       39.0        43.0        49.0
3           20.0       39.0        99.0        23.0
4           13.0        1.0         8.0        16.0
...          ...        ...         ...         ...
8198         NaN        1.0         NaN         NaN
8199         NaN        1.0         NaN         NaN
8200         NaN        1.0         NaN         NaN
8201         NaN        1.0         NaN         NaN
8202         NaN        1.0         NaN         NaN

[8203 rows x 4 columns]


In [24]:
# code minimally adapted from Doyle et al, 2016 / Yurovski et al., 2016

import operator, itertools
def group(utterances):
	utterances.sort(key=operator.itemgetter('convId'))
	list1 = []
	for key, items in itertools.groupby(utterances, operator.itemgetter('convId')):
		list1.append(list(items))
  
	return list1


def allMarkers(markers):
	categories = []
	for marker in markers:
		categories.append(marker)
	return categories

def checkMarkers(markers):
	toReturn = []
	for marker in markers:
		if isinstance(marker, str):
			toReturn.append({"marker": marker, "category": marker})
		else:
			toReturn.append(marker)
	return toReturn


def findMarkersInConvo(markers,convo):
	ba = {} # Number of times Person A and person B says the marker["marker"]
	bna = {}
	nbna = {}
	nba = {}
	for utterance in convo:				
		for j, marker in enumerate(markers):
			word = marker["marker"]
			msgMarker = word in utterance["msgMarkers"]
			replyMarker = word in utterance["replyMarkers"]
			
			if msgMarker and replyMarker:
				ba[word] = ba.get(word,0) + 1
			elif replyMarker and not msgMarker:
				bna[word] = bna.get(word,0) + 1
			elif not replyMarker and msgMarker:
				nba[word] = nba.get(word,0) + 1
			else:
				nbna[word] = nbna.get(word,0) + 1
	#		print(msgMarker, replyMarker)
			print(ba, bna, nba, nbna)
	return({'ba': ba,'bna': bna,'nba': nba,'nbna': nbna}) 
 
def metaDataExtractor(groupedUtterances, markers,corpusType=''):
	results = []
	for i, convo in enumerate(groupedUtterances):
				
		toAppend = findMarkersInConvo(markers,convo)		
		results.append(toAppend)
	return results

def readMarkers(markersFile,dialect=None):
	if dialect is None:
		reader = csv.reader(open(markersFile))
	else:
		reader = csv.reader(open(markersFile),dialect=dialect)
	markers = []
	print('marker\tcategory')
	for i, row in enumerate(reader):
		toAppend = {}
		toAppend["marker"] = row[0]
		if(len(row) > 1):
			toAppend["category"] = row[1]
		else:
			toAppend["category"] = row[0]
		markers.append(toAppend)
		#print(toAppend["marker"]+'\t'+toAppend["category"])
	return markers

def writeFile(results, outputFile, shouldWriteHeader):
	if len(results) == 0:
		print("x")
		return
	toWrite = []
	header = sorted(list(results[0].keys()))
	for row in results:
		toAppend = []
		for key in header:
			toAppend.append(row[key])
		toWrite.append(toAppend)
	if shouldWriteHeader:
		with open(outputFile, "w", newline='') as f:
			writer = csv.writer(f)
			writer.writerows([header])
		f.close()
	with open(outputFile, "a", newline='') as f:
		writer = csv.writer(f)
		writer.writerows(toWrite)
	f.close()

def determineCategories(msgMarkers,catdict,useREs=False):
	msgCats = []
	#iterate over catdict items {category: [words/REs]}
	for cd in catdict.items():
		if useREs:
			if any(any(wordre.match(marker) for marker in msgMarkers) for wordre in cd[1]):	#if REs, see if any tokens match each RE
				msgCats.append(cd[0])
		else:
			if any(word in msgMarkers for word in cd[1]):			#if just words, see if any word in category also in msg
				msgCats.append(cd[0])
	return msgCats

def makeCatDict(markers,useREs=False):
	mdict = {}
	for m in markers:
		marker = re.compile(''.join([m["marker"], '$'])) if useREs else m["marker"]
		if m["category"] in mdict:
			mdict[m["category"]].append(marker)
		else:
			mdict[m["category"]] = [marker]
		#mdict[m["category"]] = mdict.get(m["category"],[]).append(m["marker"])	#Need to swap marker and category labels
		#mdict[m["marker"]] = mdict.get(m["marker"],[]).append(m["category"])
	return(mdict)
 
def createAlignmentDict(category,result,smoothing,corpusType=''):
	toAppend = {}
	print(category)
	print("R", result)
	ba = int(result["ba"].get(category, 0))
	bna = int(result["bna"].get(category, 0))
	nbna = int(result["nbna"].get(category, 0))
	nba = int(result["nba"].get(category, 0))
	print(ba,bna,nbna,nba)
	#Calculating alignment only makes sense if we've seen messages with and without the marker
	if (((ba+nba)==0 or (bna+nbna)==0)):
		return(None)
	
	toAppend["category"] = category
		
	#Calculating Echoes of Power alignment 
	powerNum = ba
	powerDenom = ba+nba
	baseNum = ba+bna
	baseDenom = ba+nba+bna+nbna

	if(powerDenom != 0 and baseDenom != 0):
		dnmalignment = powerNum/powerDenom - baseNum/baseDenom
		toAppend["dnmalignment"] = dnmalignment
	else:
		toAppend["dnmalignment"] = False
	
	powerNum = ba
	powerDenom = ba+nba
	baseDenom = bna+nbna
	baseNum = bna
	powerProb = math.log((powerNum+smoothing)/float(powerDenom+2*smoothing))
	baseProb = math.log((baseNum+smoothing)/float(baseDenom+2*smoothing))
	alignment = powerProb - baseProb
	toAppend["alignment"] = alignment
	
	toAppend["ba"] = ba
	toAppend["bna"] = bna
	toAppend["nba"] = nba
	toAppend["nbna"] = nbna
	#print("toapp:", toAppend)
	return(toAppend)
 
def runFormula(results, markers, smoothing,corpusType):
	toReturn = []
	categories = allMarkers(markers)
	#print("h", results) 
#	print("c", categories)
	for i, result in enumerate(results):
		#print(i, result)
		for j, category in enumerate(categories):
			toAppend = createAlignmentDict(category["category"],result,smoothing,corpusType)
			print("here", toAppend)   
			if toAppend is not None:
				toReturn.append(toAppend)
	toReturn = sorted(toReturn, key=lambda k: (k["speakerId"],k["replierId"],k["category"]))
	return toReturn

In [12]:
convo_utts= list(convo.iter_utterances()) 
convo_utts

[Utterance({'obj_type': 'utterance', 'meta': {'score': 2, 'top_level_comment': None, 'retrieved_on': 1510457299, 'gilded': 0, 'gildings': None, 'subreddit': 'Cornell', 'stickied': False, 'permalink': '/r/Cornell/comments/77khqw/does_the_loft_need_a_fake_to_enter/', 'author_flair_text': '', 'parsed': [{'rt': 1, 'toks': [{'tok': '[', 'tag': '-LRB-', 'dep': 'punct', 'pos': 'PUNCT', 'lemma': '[', 'up': 1, 'dn': []}, {'tok': 'deleted', 'tag': 'VBN', 'dep': 'ROOT', 'pos': 'VERB', 'lemma': 'delete', 'dn': [0, 2]}, {'tok': ']', 'tag': '-RRB-', 'dep': 'punct', 'pos': 'PUNCT', 'lemma': ']', 'up': 1, 'dn': []}]}]}, 'vectors': [], 'speaker': Speaker({'obj_type': 'speaker', 'meta': {}, 'vectors': [], 'owner': <convokit.model.corpus.Corpus object at 0x7f1e3862f9d0>, 'id': '[deleted]'}), 'conversation_id': '77khqw', 'reply_to': None, 'timestamp': 1508482721, 'text': '[deleted]', 'owner': <convokit.model.corpus.Corpus object at 0x7f1e3862f9d0>, 'id': '77khqw'}),
 Utterance({'obj_type': 'utterance', 'm

In [13]:
import pprint

# pair utterances from convokit 
markers = ["the", "of", "very", "to", "through", "and","each", "other", "'d", "more", "less", "front", "back", "please", "any", "due", "just", "a", "in", "that", "for", "on", "are", "you", "it", "was", "were", "I", "as", "with", "they", "be", "at", "too", "have", "does", "this", "from", "or", "had", "by", "but", "some", "also", "what", "can", "out", "all", "your", "my", "up", "even", "so", "yes", "when", "almost", "no", "must", "should", "will", "would", "not", "a", "that", "at"]


utterances =[]
previous_utt = dict()
#convo = corpus.random_conversation()

convo_utts= list(convo.iter_utterances()) 


for i in convo.traverse('bfs'): #iterate through utterances in tree-like (breadth) structure, level by level
  #print(i)
  d = {} #each pair of speakers is a 'd' dictionary
  previous_utt[i.id] = (i.text,i.speaker.id)
  print(previous_utt)
  if i.reply_to == None:  #ignore if no reply
    continue
  d['convId'] = (previous_utt[i.reply_to][1], i.speaker.id)
  print(d['convId'])
  d['speakerId'] = previous_utt[i.reply_to][1] #?
  print(d['speakerId'])
  d['msg'] = previous_utt[i.reply_to][0]
  print(d['msg'])
  d['reply'] = i.text
  print(d['reply'])
  d['msgMarkers'] = []
  print(d['msgMarkers'])
  d['replyMarkers'] = []
  d['msgTokens']= []
  print(d['msgTokens'])
  d['replyTokens']= []
  d["replierId"] = i.speaker.id
  print(d['replierId'])
 # pprint.pprint(d)
  utterances.append(d)



{'77khqw': ('[deleted]', '[deleted]')}
{'77khqw': ('[deleted]', '[deleted]'), 'domyj1g': ('Probably not. They card at the door, not at the bar, so I’m guessing they won’t let a minor in at all.', 'giottoblue')}
('[deleted]', 'giottoblue')
[deleted]
[deleted]
Probably not. They card at the door, not at the bar, so I’m guessing they won’t let a minor in at all.
[]
[]
giottoblue
{'77khqw': ('[deleted]', '[deleted]'), 'domyj1g': ('Probably not. They card at the door, not at the bar, so I’m guessing they won’t let a minor in at all.', 'giottoblue'), 'donalum': ("They often card at the door. Sometimes they'll card at the bar, but I think that's when it's not as crowded (e.g. early on Wednesday evening). If you're asking for tonight, they'll probably card at the door.", 'Clue_Balls')}
('[deleted]', 'Clue_Balls')
[deleted]
[deleted]
They often card at the door. Sometimes they'll card at the bar, but I think that's when it's not as crowded (e.g. early on Wednesday evening). If you're asking for

In [14]:
for idx, utt in enumerate(utterances):  #append marker in utterance metadata if it exists
  for marker in markers:
			if marker in utt['msg']:
				utterances[idx]['msgMarkers'].append(marker)
			if marker in utt['reply']:
				utterances[idx]['replyMarkers'].append(marker)



In [ ]:
pprint.pprint(utterances)

In [ ]:
#main function - run alignment
import math, csv

smoothing=1
shouldWriteHeader=True
outputFile='aaaaaaaaa'
markers = checkMarkers(markers)
groupedUtterances = group(utterances)
#print(groupedUtterances)
metaData = metaDataExtractor(groupedUtterances,markers) 
#print(metaData)        


In [27]:
results = runFormula(metaData, markers, smoothing,'')

print(results)

writeFile(results, outputFile, shouldWriteHeader)


0 {'ba': {}, 'bna': {'the': 1, 'of': 1, 'to': 1, 'a': 2, 'in': 1, 'that': 2, 'for': 1, 'on': 1, 'you': 1, 'it': 1, 'I': 1, 'as': 1, 'they': 1, 'at': 2, 'or': 1, 'but': 1, 'even': 1, 'when': 1, 'no': 1, 'not': 1}, 'nba': {}, 'nbna': {'very': 1, 'through': 1, 'and': 1, 'each': 1, 'other': 1, "'d": 1, 'more': 1, 'less': 1, 'front': 1, 'back': 1, 'please': 1, 'any': 1, 'due': 1, 'just': 1, 'are': 1, 'was': 1, 'were': 1, 'with': 1, 'be': 1, 'too': 1, 'have': 1, 'does': 1, 'this': 1, 'from': 1, 'had': 1, 'by': 1, 'some': 1, 'also': 1, 'what': 1, 'can': 1, 'out': 1, 'all': 1, 'your': 1, 'my': 1, 'up': 1, 'so': 1, 'yes': 1, 'almost': 1, 'must': 1, 'should': 1, 'will': 1, 'would': 1}}
the
R {'ba': {}, 'bna': {'the': 1, 'of': 1, 'to': 1, 'a': 2, 'in': 1, 'that': 2, 'for': 1, 'on': 1, 'you': 1, 'it': 1, 'I': 1, 'as': 1, 'they': 1, 'at': 2, 'or': 1, 'but': 1, 'even': 1, 'when': 1, 'no': 1, 'not': 1}, 'nba': {}, 'nbna': {'very': 1, 'through': 1, 'and': 1, 'each': 1, 'other': 1, "'d": 1, 'more': 1, 